In [2]:
import csv
import pandas as pd
# Increase the maximum field size limit
csv.field_size_limit(10**8)
import psycopg2
from psycopg2 import sql

In [2]:
def split_list_into_parts(input_list, part_size):
    return [input_list[i:i + part_size] for i in range(0, len(input_list), part_size)]


In [ ]:
# Specify the path to your CSV file
file_path = '../data/20181024_d1_0830_0900.csv'

# Open the CSV file
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    i = 0
    # Read each row one by one
    for line_number, row in enumerate(csv_reader):
        # Process individual lines as needed
        if(i == 10):
            break
        print(i)
        if (i == 0):
            # create the column names for the fact and dimensional table
            columns = row[0]
            dimension_columns = [columns.split(";")[0]] + columns.split(";")[4:]
            
            # create fact and dimensional table
            fact_df = pd.DataFrame(columns=columns.split(";")[:4])
            dimension_df = pd.DataFrame(columns=dimension_columns)          
        else:
            data = [elt  for elt in row[0].split(";") if elt != " "]
            track_id = data[0]
            
            fact_df.loc[len(fact_df)] = data[:4]
            
            # Split the list into parts containing 6 elements each
            split_parts = split_list_into_parts(data[4:], 6)

            for dimensions in split_parts:
                dimensions.insert(0, track_id)
                dimension_df.loc[len(dimension_df)] = dimensions
        i+=1     
        fact_df.to_csv("../data/fact_df.csv", index=False)   

In [ ]:
print(fact_df.shape)
print(dimension_df.shape)

In [ ]:
fact_df.track_id.unique()

In [ ]:

# Define the paths to your CSV files
fact_csv_path = '../data/fact_df.csv'  # Change this to your actual CSV path
dimensional_csv_path = '../data/dimension_df.csv'  # Change this to your actual CSV path

# Read the CSV files into DataFrames
fact_df = pd.read_csv(fact_csv_path)
dimensional_df = pd.read_csv(dimensional_csv_path)

# Connect to the database
conn = psycopg2.connect(
    
            host="172.31.98.138",
            database="airflow_traffic_db",
            port="5432",
            user="postgres",
            password="password"
        )
cursor = conn.cursor()
print(fact_df.columns)
print(dimensional_df.columns)

# Insert data into fact_df
for index, row in fact_df.iterrows():
    cursor.execute("""
        INSERT INTO fact_df (track_id, type, traveled_d, avg_speed) 
        VALUES (%s, %s, %s, %s)
    """, (row['track_id'], row['type'], row['traveled_d'], row['avg_speed']))

# Insert data into dimensional_df
for index, row in dimensional_df.iterrows():
    cursor.execute("""
        INSERT INTO dimensional_df (track_id, lat, lon, speed, ion_acc, lat_acc, time) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (row['track_id'], row['lat'], row['lon'], row['speed'], row['ion_acc'], row['lat_acc'], row['time']))

# Commit the changes and close the connection
conn.commit()
cursor.close()
conn.close()


In [ ]:
dimension_df